<a href="https://colab.research.google.com/github/danielecaliari/AML/blob/main/modelli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + 'progetto_AML/'

Mounted at /content/drive


In [3]:
from scipy import sparse
train = sparse.load_npz(base_dir + 'train.npz')
test = sparse.load_npz(base_dir + 'test.npz')


In [33]:
from scipy import sparse
label = pd.read_csv(base_dir + 'target_label.csv')
label = sparse.csr_matrix(label['price'])
label.shape

(1, 1481661)

In [22]:
def create_mlp(dim):
	# define our MLP network
  model = Sequential()
  model.add(Dense(8, input_dim=dim, activation="relu"))
  model.add(Dense(4, activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

Split in training & validation

In [23]:

(train, validation, label_train, label_validation) = train_test_split(train , label , test_size=0.25, random_state=42)

ValueError: ignored

In [ ]:
validation

<370416x45811 sparse matrix of type '<class 'numpy.float64'>'
	with 12798615 stored elements in Compressed Sparse Row format>

In [9]:
train.shape[1]

121534

In [12]:
model = create_mlp(train.shape[1])
model.compile(loss="mean_absolute_percentage_error")
model.fit(x=train, y=label_train, validation_data=(validation, label_validation), epochs=200, batch_size=8)

ValueError: ignored

Prediction

In [ ]:
preds = model.predict(test)
# fare inversa log